In [203]:
import requests
import re
import json
from pprint import pprint
from shapely import wkt
from shapely.geometry import Polygon
from datetime import datetime, date, timezone

In [3]:
CMR_URL = 'https://cmr.earthdata.nasa.gov/search/granules.json'

wkt_poly="POLYGON((-148.1418 64.6321,-147.4753 64.6321,-147.4753 64.9466,-148.1418 64.9466,-148.1418 64.6321))"


In [ ]:
class product:
    

In [80]:
def format_points(point_string):
    converted_to_float = [float(x) for x in point_string.split(' ')]
    points = [list(t) for t in zip(converted_to_float[1::2], converted_to_float[::2])]
    return points

In [220]:
def product_time(product_name):
    product_time_regex = re.compile(
        r"S.*1SDV_(?P<start_year>\d{4})(?P<start_month>\d{2})(?P<start_day>\d{2})T(?P<start_hour>\d{2})("
        r"?P<start_minute>\d{2})(?P<start_second>\d{2})_(?P<end_year>\d{4})(?P<end_month>\d{2})(?P<end_day>\d{2})T("
        r"?P<end_hour>\d{2})(?P<end_minute>\d{2})(?P<end_second>\d{2})_*")

    regex_match = re.match(product_time_regex, product_name)
    time_dict = regex_match.groupdict()

    # converts all dates/times values in dictionary from int to string
    for k, v in time_dict.items():
        time_dict[k] = int(v)

    start = datetime(time_dict["start_year"], time_dict["start_month"], time_dict["start_day"],
                     time_dict["start_hour"], time_dict["start_minute"], time_dict["start_second"])

    end = datetime(time_dict["end_year"], time_dict["end_month"], time_dict["end_day"],
                   time_dict["end_hour"], time_dict["end_minute"], time_dict["end_second"])


    return start, end

In [221]:
def product_in_time_bounds(product_name, start, end):
    prod_start, prod_end = product_time(product_name)
    
    return prod_start > start and prod_end < end
    

In [222]:
def format_time(time: datetime):
#     utc_time = time.astimezone(timezone.utc)
#     return utc_time.isoformat(timespec='seconds')
    return time.isoformat()

In [223]:
test_date = datetime(2020,8,1,0,0,0)
print(format_time(test_date))

2020-08-01T00:00:00


In [224]:
def get_cmr_metadata(polygon, start, end):
    payload = {}
    
    payload['provider'] = 'ASF'
    payload['temporal'] = (start, end)
    payload['polygon'] = polygon
    payload['short_name'] = (
            'SENTINEL-1A_DP_GRD_HIGH',
            'SENTINEL-1B_DP_GRD_HIGH',
            )
    payload['page_size'] = 100


    response = requests.post(CMR_URL, data=payload)
    granules = [
        {
            'name': entry['producer_granule_id'],
#             'polygon': Polygon(format_points(entry['polygons'][0][0]))
        } for entry in response.json()['feed']['entry']
    ]
#     response = requests.post(CMR_URL, data=cmr_parameters)
    response.raise_for_status()
    return granules

In [225]:

input_polygon = "-148.2421,63.9554,-146.4533,63.9554,-146.4533,64.9447,-148.2421,64.9447,-148.2421,63.9554"
start_time = format_time(datetime(2020,8,day=1,hour=0,minute=0,second=0))
# print(start_time)
end_time = format_time(datetime(2020,8,day=8,hour=0,minute=0,second=0))

meta_data = get_cmr_metadata(input_polygon,end_time, start_time)
# for item in meta_data:
#     pprint(item)

# for name in meta_data:
#     print(name['name'])
    
name_list = [name['name'] for name in meta_data]
# pprint(f"{set(name_list)} name_list COUNT={len(name_list)} set COUNT={len(set(name_list))}")

start_time = datetime(2020,8,day=1,hour=0,minute=0,second=0)
end_time = datetime(2020,8,day=8,hour=0,minute=0,second=0)

for name in name_list:
    if product_in_time_bounds(name, start_time, end_time):
        print(name+',')
    


S1A_IW_GRDH_1SDV_20200807T162818_20200807T162843_033803_03EB45_A6DF,
S1A_IW_GRDH_1SDV_20200807T162843_20200807T162908_033803_03EB45_30A8,


In [ ]:
# test case for wkt_to_polyString


test_wkt="POLYGON((-148.2421 63.9554,-146.4533 63.9554,-146.4533 64.9447,-148.2421 64.9447,-148.2421 63.9554))"


expected_str = "-148.2421,63.9554,-146.4533,63.9554,-146.4533,64.9447,-148.2421,64.9447,-148.2421,63.9554"